# Chapter 10: Applications of Metamaterials

**PyMetaLearn - Metamaterial Physics Workbook**

---

## Learning Objectives

By the end of this chapter, you will:
- Understand the principles behind electromagnetic cloaking
- Learn about perfect absorbers and their applications
- Explore flat optics: metalenses and holographic metasurfaces
- Appreciate real-world applications in sensing, communications, and imaging
- See the future directions of metamaterial research

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle, Wedge, FancyArrowPatch
from matplotlib.collections import PatchCollection
from IPython.display import display, Markdown, Math

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

# Physical constants
c = 3e8            # Speed of light (m/s)
epsilon_0 = 8.854e-12
mu_0 = 4 * np.pi * 1e-7

print("Libraries loaded!")

---

## 10.1 Electromagnetic Cloaking

### The Dream of Invisibility

One of the most exciting applications of metamaterials is **cloaking** — bending light around an object so that it becomes invisible.

### Transformation Optics (Pendry & Leonhardt, 2006)

The key insight: Maxwell's equations are **form-invariant** under coordinate transformations.

If we apply a coordinate transformation $x \to x'$, the fields see an **effective material** with:

$$\varepsilon'_{ij} = \frac{\det(\Lambda)^{-1} \cdot \Lambda_{ik} \cdot \Lambda_{jl} \cdot \varepsilon_{kl}}{\det(\Lambda)}$$

Where $\Lambda = \partial x' / \partial x$ is the Jacobian of the transformation.

### The Cloak Prescription

To make a spherical region invisible:
1. **Compress** the region $r < b$ to $r < a$ (where $a < b$)
2. Fill the shell $a < r < b$ with the required metamaterial
3. Light bends around the "hole" at $r < a$

In [ ]:
# Visualize cloaking concept
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Ray trajectories without cloak
ax1 = axes[0]
for y0 in np.linspace(-1.5, 1.5, 15):
    ax1.plot([-2, 2], [y0, y0], 'b-', linewidth=0.8)
ax1.add_patch(Circle((0, 0), 0.5, facecolor='gray', edgecolor='black', linewidth=2))
ax1.set_xlim(-2, 2)
ax1.set_ylim(-2, 2)
ax1.set_aspect('equal')
ax1.set_title('Without Cloak:\nWaves scattered by object', fontsize=12)
ax1.axis('off')

# Ray trajectories with cloak
ax2 = axes[1]

def cloak_trajectory(y0, r_inner=0.5, r_outer=1.5):
    """Calculate ray path through a cylindrical cloak."""
    x = np.linspace(-2, 2, 200)
    y = np.zeros_like(x)
    
    for i, xi in enumerate(x):
        r = np.sqrt(xi**2 + y0**2)
        if r < r_outer and abs(xi) < r_outer:
            # Inside cloak region - bend around
            if r < r_inner:
                y[i] = np.nan  # Inside cloaked region
            else:
                # Smooth bending
                t = (r - r_inner) / (r_outer - r_inner)
                scale = r_inner + (r_outer - r_inner) * t
                y[i] = y0 * scale / r if r > 0.1 else y0
        else:
            y[i] = y0
    
    return x, y

for y0 in np.linspace(-1.8, 1.8, 15):
    if abs(y0) < 0.5:
        continue  # Skip rays that would hit the object
    x, y = cloak_trajectory(y0)
    ax2.plot(x, y, 'b-', linewidth=0.8)

# Draw cloak shell
theta = np.linspace(0, 2*np.pi, 100)
ax2.fill(1.5*np.cos(theta), 1.5*np.sin(theta), alpha=0.2, color='purple')
ax2.add_patch(Circle((0, 0), 0.5, facecolor='gray', edgecolor='black', linewidth=2))
ax2.set_xlim(-2, 2)
ax2.set_ylim(-2, 2)
ax2.set_aspect('equal')
ax2.set_title('With Cloak:\nWaves bend around object', fontsize=12)
ax2.axis('off')

# Material requirements
ax3 = axes[2]
r = np.linspace(0.5, 1.5, 100)
r_inner = 0.5
r_outer = 1.5

# For cylindrical cloak:
eps_r = ((r - r_inner) / r)**2
eps_theta = 1
mu_z = ((r_outer / (r_outer - r_inner))**2) * ((r - r_inner) / r)**2

ax3.plot(r, eps_r, 'b-', linewidth=2, label=r'$\varepsilon_r$ (radial)')
ax3.plot(r, mu_z, 'r-', linewidth=2, label=r'$\mu_z$')
ax3.axhline(y=1, color='gray', linestyle='--')
ax3.axvline(x=0.5, color='gray', linestyle=':', label='Inner radius')
ax3.axvline(x=1.5, color='gray', linestyle=':', label='Outer radius')

ax3.set_xlabel('Radial position r', fontsize=12)
ax3.set_ylabel('Material parameters', fontsize=12)
ax3.set_title('Required Material Profile\n(Anisotropic!)', fontsize=12)
ax3.legend(fontsize=9)
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Key insight: Cloaking requires ANISOTROPIC, spatially-varying materials")
print("with both ε and μ parameters that range from 0 to >>1.")

---

## 10.2 Cloaking: Challenges and Reality

### The Challenges

1. **Extreme material parameters**: Need ε, μ ranging from 0 to ∞
2. **Anisotropy**: All directions must be independently controlled
3. **Narrowband**: Metamaterials work at specific frequencies
4. **Losses**: Any absorption breaks the cloak

### Experimental Demonstrations

| Year | Type | Notes |
|------|------|-------|
| 2006 | 2D microwave | First demo (Duke) |
| 2012 | 3D microwave | Larger objects |
| 2014 | Carpet cloak (optical) | Conceals bumps |
| 2015+ | Thin-film and metasurface cloaks | More practical |

---

## 10.3 Perfect Absorbers

A **perfect absorber** has:
- Zero reflection: $R = 0$
- Zero transmission: $T = 0$
- All energy absorbed: $A = 1$

### The Recipe

1. **Impedance matching**: $Z = Z_0 = \sqrt{\mu/\varepsilon} = 377\Omega$
2. **High loss**: Strong absorption once light enters
3. **Back reflector**: Prevents transmission

### Typical Structure

Metal-Insulator-Metal (MIM):
- Top: Patterned metal (metasurface)
- Middle: Thin dielectric spacer
- Bottom: Metal ground plane

In [ ]:
# Perfect absorber design
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Structure visualization
ax1 = axes[0]

# Ground plane
ax1.add_patch(Rectangle((-2, 0), 4, 0.3, facecolor='gold', edgecolor='black'))
ax1.text(-2.3, 0.15, 'Metal\nground', fontsize=10, va='center', ha='right')

# Dielectric spacer
ax1.add_patch(Rectangle((-2, 0.3), 4, 0.5, facecolor='lightblue', edgecolor='black'))
ax1.text(-2.3, 0.55, 'Dielectric\nspacer', fontsize=10, va='center', ha='right')

# Patterned top metal (patches)
for x in np.linspace(-1.5, 1.5, 4):
    ax1.add_patch(Rectangle((x-0.3, 0.8), 0.6, 0.15, facecolor='gold', edgecolor='black'))
ax1.text(-2.3, 0.85, 'Metal\npatches', fontsize=10, va='center', ha='right')

# Incident wave
ax1.annotate('', xy=(0, 1.5), xytext=(-1, 2.3),
            arrowprops=dict(arrowstyle='->', color='red', lw=2))
ax1.text(-0.8, 2, 'Incident', fontsize=11, rotation=-45)

# Absorbed (no reflection)
ax1.text(0.5, 1.5, 'No reflection!\n(R ≈ 0)', fontsize=11, color='green')

ax1.set_xlim(-2.5, 2.5)
ax1.set_ylim(-0.3, 2.5)
ax1.set_aspect('equal')
ax1.set_title('Perfect Absorber Structure', fontsize=14)
ax1.axis('off')

# Absorption spectrum
ax2 = axes[1]

# Model Lorentzian absorption
f = np.linspace(1, 20, 500)  # GHz or THz
f0 = 10  # Resonance
gamma = 0.5

# Absorption profile
A = 1 / (1 + ((f - f0) / gamma)**2)
R = 0.02 + 0.1 * (1 - A)  # Small residual reflection
T = 0.0 * np.ones_like(f)  # Zero transmission

ax2.plot(f, A * 100, 'b-', linewidth=2, label='Absorption')
ax2.plot(f, R * 100, 'r--', linewidth=2, label='Reflection')
ax2.plot(f, T * 100, 'g:', linewidth=2, label='Transmission')

ax2.axhline(y=99, color='gray', linestyle='--', alpha=0.5)
ax2.text(15, 95, '>99% absorption', fontsize=10)

ax2.set_xlabel('Frequency (GHz or THz)', fontsize=14)
ax2.set_ylabel('Percentage (%)', fontsize=14)
ax2.set_title('Perfect Absorber Spectrum', fontsize=14)
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_xlim(1, 20)
ax2.set_ylim(0, 105)

plt.tight_layout()
plt.show()

print("Applications: Thermal emitters, bolometers, stealth, solar absorbers")

---

## 10.4 Flat Optics: Metalenses

Traditional lenses rely on **curvature** to focus light. Metalenses use **phase gradients**.

### The Concept

A flat metasurface with position-dependent phase:

$$\phi(r) = -\frac{2\pi}{\lambda}\left(\sqrt{r^2 + f^2} - f\right)$$

Where:
- $r$ = distance from center
- $f$ = desired focal length
- This is a **hyperbolic** phase profile

### Advantages Over Traditional Lenses

| Metalens | Traditional |
|----------|-------------|
| Flat, thin (< λ) | Curved, thick |
| No spherical aberration | Inherent aberration |
| Easy integration | Bulky |
| Multi-functional | Single function |

In [ ]:
# Metalens focusing
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Phase profile
ax1 = axes[0]

r = np.linspace(-5, 5, 500)
f = 10  # Focal length
wavelength = 0.5

# Hyperbolic phase profile
phase = -(2 * np.pi / wavelength) * (np.sqrt(r**2 + f**2) - f)
phase_wrapped = np.mod(phase, 2 * np.pi)  # Wrap to [0, 2π]

ax1.plot(r, phase, 'b-', linewidth=2, label='Ideal phase')
ax1.plot(r, phase_wrapped, 'r-', linewidth=1.5, alpha=0.7, label='Wrapped (0 to 2π)')

ax1.set_xlabel('Position r (μm)', fontsize=14)
ax1.set_ylabel('Phase (radians)', fontsize=14)
ax1.set_title('Metalens Phase Profile', fontsize=14)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Ray tracing
ax2 = axes[1]

# Metalens surface
ax2.plot([-3, 3], [0, 0], 'b-', linewidth=4, label='Metalens')

# Incident plane wave (rays)
for y in np.linspace(-2.5, 2.5, 11):
    ax2.plot([y, y], [3, 0], 'g-', linewidth=1, alpha=0.5)
    ax2.annotate('', xy=(0, -5), xytext=(y, 0),
                arrowprops=dict(arrowstyle='->', color='red', lw=1))

# Focal point
ax2.plot(0, -5, 'ro', markersize=15, label='Focus')
ax2.text(0.3, -5, f'f = {f}', fontsize=12)

ax2.set_xlim(-4, 4)
ax2.set_ylim(-7, 4)
ax2.set_aspect('equal')
ax2.set_title('Metalens Focusing', fontsize=14)
ax2.set_xlabel('x (μm)', fontsize=12)
ax2.set_ylabel('z (μm)', fontsize=12)
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Modern metalenses achieve diffraction-limited focusing with")
print("thickness < 1 μm — revolutionizing compact optics!")

---

## 10.5 Holographic Metasurfaces

Metasurfaces can encode **complex holographic images**:

### Concept

1. **Calculate** the required phase/amplitude at each pixel
2. **Design** meta-atoms to achieve those values
3. **Fabricate** the 2D pattern
4. **Illuminate** with coherent light → hologram appears!

### Advantages

- Compact: single thin layer
- High efficiency: >80% possible
- Multi-functional: wavelength, polarization, angle dependent

In [ ]:
# Simple hologram simulation
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Target image (simple letter)
ax1 = axes[0]
N = 100
target = np.zeros((N, N))

# Draw "M" for Metamaterial
target[20:80, 20:30] = 1  # Left stroke
target[20:80, 70:80] = 1  # Right stroke
target[20:40, 30:50] = 1  # Left diagonal
target[20:40, 50:70] = 1  # Right diagonal

ax1.imshow(target, cmap='hot', origin='lower')
ax1.set_title('Target Image', fontsize=14)
ax1.axis('off')

# Calculate hologram phase (Gerchberg-Saxton simplified)
ax2 = axes[1]

# FFT to get Fourier plane
target_ft = np.fft.fftshift(np.fft.fft2(target))
hologram_phase = np.angle(target_ft)

ax2.imshow(hologram_phase, cmap='twilight', origin='lower')
ax2.set_title('Metasurface Phase Pattern', fontsize=14)
ax2.axis('off')

# Reconstruct
ax3 = axes[2]

# Apply phase-only hologram
reconstruction = np.fft.ifft2(np.fft.ifftshift(np.exp(1j * hologram_phase)))
intensity = np.abs(reconstruction)**2

ax3.imshow(intensity, cmap='hot', origin='lower')
ax3.set_title('Reconstructed Image', fontsize=14)
ax3.axis('off')

plt.tight_layout()
plt.show()

print("This is a simplified simulation. Real metasurface holograms")
print("use sophisticated optimization for high-fidelity images.")

---

## 10.6 Sensing Applications

Metamaterials enable ultra-sensitive **sensors**:

### Refractive Index Sensing

- Resonance frequency shifts with local refractive index
- Can detect single molecules!

### Surface Enhanced Spectroscopy

- Plasmonic "hot spots" enhance local fields by 10²-10⁴
- SERS: Surface Enhanced Raman Scattering
- SEIRA: Surface Enhanced Infrared Absorption

### Typical Sensitivities

| Platform | Sensitivity |
|----------|-------------|
| SPR sensor | ~100 nm/RIU |
| Metamaterial sensor | ~500 nm/RIU |
| Fano resonance | ~1000 nm/RIU |

In [ ]:
# Sensing demonstration
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Resonance shift with refractive index
ax1 = axes[0]

freq = np.linspace(1, 15, 500)

# Different refractive indices
n_values = [1.0, 1.1, 1.2, 1.33]  # Air, protein, cell, water
labels = ['Air (n=1.0)', 'Protein layer', 'Cell', 'Water (n=1.33)']
colors = ['blue', 'green', 'orange', 'red']

f0_base = 10
sensitivity = 5  # GHz per 0.1 RIU

for n, label, color in zip(n_values, labels, colors):
    f0 = f0_base - sensitivity * (n - 1)
    transmission = 1 - 0.9 / (1 + ((freq - f0) / 0.5)**2)
    ax1.plot(freq, transmission * 100, color=color, linewidth=2, label=label)

ax1.set_xlabel('Frequency (GHz or THz)', fontsize=14)
ax1.set_ylabel('Transmission (%)', fontsize=14)
ax1.set_title('Metamaterial Sensor: Resonance Shift', fontsize=14)
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)

# Sensitivity curve
ax2 = axes[1]

n_range = np.linspace(1.0, 1.5, 50)
f_shift = sensitivity * (n_range - 1) * 10  # Convert to total shift
wavelength_shift = f_shift * 30  # Approximate conversion to nm

ax2.plot(n_range, wavelength_shift, 'b-', linewidth=2)
ax2.scatter([1.0, 1.1, 1.2, 1.33], [0, 150, 300, 495], c=colors, s=100, zorder=5)

ax2.set_xlabel('Refractive Index', fontsize=14)
ax2.set_ylabel('Resonance Shift (nm)', fontsize=14)
ax2.set_title('Sensitivity: ~500 nm/RIU', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Metamaterial sensors can detect:")
print("• Protein binding (sub-monolayer)")
print("• DNA hybridization")
print("• Gas molecules")
print("• Cancer biomarkers")

---

## 10.7 Communications and Antennas

### Reconfigurable Intelligent Surfaces (RIS)

Large metasurfaces that can:
- **Reflect** signals toward users
- **Focus** beams in real-time
- **Cancel** interference

Key technology for **6G** wireless!

### Metamaterial Antennas

- Leaky-wave antennas: beam steering by frequency
- Epsilon-near-zero (ENZ) antennas: supercoupling
- Miniaturized antennas: using slow-wave structures

In [ ]:
# Beam steering visualization
fig, ax = plt.subplots(figsize=(12, 8))

# Metasurface
ax.plot([-4, 4], [0, 0], 'b-', linewidth=6, label='Metasurface')

# Draw phase-gradient elements
for x in np.linspace(-3.5, 3.5, 15):
    phase = (x + 3.5) / 7 * 2 * np.pi  # Linear phase gradient
    color = plt.cm.twilight(phase / (2 * np.pi))
    ax.add_patch(Rectangle((x - 0.2, -0.15), 0.4, 0.3, 
                           facecolor=color, edgecolor='black'))

# Incident wave (normal)
for x in np.linspace(-3, 3, 7):
    ax.annotate('', xy=(x, 0.3), xytext=(x, 3),
               arrowprops=dict(arrowstyle='->', color='green', lw=1.5))
ax.text(-4, 2, 'Incident\n(normal)', fontsize=12, color='green')

# Reflected wave (steered)
steer_angle = 30  # degrees
for x in np.linspace(-3, 3, 7):
    dx = -2 * np.sin(np.radians(steer_angle))
    dy = -2 * np.cos(np.radians(steer_angle))
    ax.annotate('', xy=(x + dx, dy), xytext=(x, -0.3),
               arrowprops=dict(arrowstyle='->', color='red', lw=1.5))

ax.text(2, -2.5, f'Steered\n({steer_angle}° off normal)', fontsize=12, color='red')

# Angle indicator
arc_theta = np.linspace(np.radians(270), np.radians(270 - steer_angle), 30)
ax.plot(0.8 * np.cos(arc_theta), 0.8 * np.sin(arc_theta), 'm-', linewidth=2)
ax.text(0.3, -0.8, f'θ = {steer_angle}°', fontsize=11, color='purple')

ax.set_xlim(-5, 5)
ax.set_ylim(-4, 4)
ax.set_aspect('equal')
ax.set_title('Beam Steering with Metasurface', fontsize=16)
ax.set_xlabel('Position (a.u.)', fontsize=12)
ax.set_ylabel('Height (a.u.)', fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("The phase gradient across the metasurface steers the reflected beam.")
print("This is the basis for reconfigurable intelligent surfaces (RIS).")

---

## 10.8 Emerging Applications

### Thermal Management
- Radiative cooling: emit IR through atmospheric window
- Thermophotovoltaics: narrow-band thermal emission

### Quantum Technologies
- Enhanced light-matter coupling
- Purcell effect engineering
- Single-photon sources

### Acoustic Metamaterials
- Sound focusing and cloaking
- Noise cancellation
- Ultrasound imaging

### Mechanical Metamaterials
- Negative Poisson's ratio (auxetics)
- Programmable stiffness
- Origami-inspired designs

In [ ]:
# Application summary visualization
fig, ax = plt.subplots(figsize=(14, 8))

applications = [
    ('Cloaking', 'Defense, Privacy', 0.3, 0.8),
    ('Perfect\nAbsorbers', 'Solar, Thermal,\nStealth', 0.3, 0.5),
    ('Metalenses', 'Cameras, AR/VR,\nMicroscopy', 0.7, 0.8),
    ('Holograms', 'Displays,\nSecurity', 0.7, 0.5),
    ('Sensors', 'Biomedical,\nEnvironmental', 0.5, 0.65),
    ('RIS/6G', 'Wireless\nCommunications', 0.5, 0.35),
    ('Thermal', 'Cooling,\nEnergy', 0.2, 0.2),
    ('Quantum', 'Computing,\nSensing', 0.8, 0.2),
]

colors = plt.cm.tab10(np.linspace(0, 1, len(applications)))

for (name, desc, x, y), color in zip(applications, colors):
    circle = Circle((x, y), 0.12, facecolor=color, edgecolor='black', alpha=0.7)
    ax.add_patch(circle)
    ax.text(x, y + 0.01, name, fontsize=10, ha='center', va='center', fontweight='bold')
    ax.text(x, y - 0.18, desc, fontsize=8, ha='center', va='top', style='italic')

# Central hub
ax.add_patch(Circle((0.5, 0.5), 0.08, facecolor='white', edgecolor='black', linewidth=2))
ax.text(0.5, 0.5, 'META\nMATERIALS', fontsize=9, ha='center', va='center', fontweight='bold')

# Connect to hub
for (name, desc, x, y), color in zip(applications, colors):
    dx = 0.5 - x
    dy = 0.5 - y
    length = np.sqrt(dx**2 + dy**2)
    ax.plot([x + 0.12 * dx/length, 0.5 - 0.08 * dx/length], 
           [y + 0.12 * dy/length, 0.5 - 0.08 * dy/length],
           'k-', alpha=0.3, linewidth=1)

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_aspect('equal')
ax.set_title('Metamaterial Applications Landscape', fontsize=18)
ax.axis('off')

plt.tight_layout()
plt.show()

---

## 10.9 Summary & Key Takeaways

### Applications Covered

1. **Cloaking**: Bending light around objects using transformation optics
2. **Perfect Absorbers**: Near-100% absorption for energy and sensing
3. **Metalenses**: Flat, thin lenses with aberration correction
4. **Holograms**: Phase-only metasurface holograms
5. **Sensors**: Ultra-sensitive detection using resonance shifts
6. **Communications**: Reconfigurable surfaces for 6G

### The State of the Field

| Application | Maturity | Commercial? |
|-------------|----------|-------------|
| Absorbers | High | Yes |
| Metalenses | High | Yes (startups) |
| Sensors | Medium | Some |
| RIS/Antennas | Medium | Emerging |
| Cloaking | Low | No |

### What's Next?

- **Active/tunable** metamaterials (using MEMS, LC, phase-change)
- **AI-designed** metamaterials (inverse design)
- **Integrated** systems (on-chip metalenses)
- **Quantum** metamaterials

---

## 📝 Final Exercises

1. **Design** a phase profile for a metalens with f = 1 mm at λ = 600 nm
2. **Calculate** the impedance match condition for a perfect absorber
3. **Explain** why broadband cloaking is fundamentally impossible
4. **Research**: Find a commercial product that uses metasurface technology
5. **Propose** a novel application for metamaterials not covered in this chapter

---

## 🎓 Congratulations!

You've completed **PyMetaLearn** — from Maxwell's equations to cutting-edge applications!

### Your Journey

1. ✅ Electromagnetic foundations
2. ✅ Material models (Drude, Lorentz)
3. ✅ Negative index physics
4. ✅ The Pendry superlens
5. ✅ Metamaterial building blocks (SRRs, wires)
6. ✅ Optical metamaterials
7. ✅ Metasurfaces
8. ✅ Real-world applications

### Further Learning

- **Review articles**: Zheludev & Kivshar (2012), Yu & Capasso (2014)
- **Textbooks**: Maier (Plasmonics), Engheta & Ziolkowski (Metamaterials)
- **Conferences**: Metamaterials, CLEO, PIERS

**Happy researching!** 🚀

---

**← [09 - Metasurfaces](09_metasurfaces.ipynb) | [Back to Start](01_foundations.ipynb) ←**